# Unscrambling crossed signals

* <https://adventofcode.com/2021/day/8>

This one is much easier than it may look, as there are only 36 different possible ways to match a given set of crossed wires to a 7-segment display: the signals for 1, 4, 7 and 8 are unambiguous, and there remaining numbers come in two groups of 3 (2, 3 and 5 light up 5 segments, and 0, 6 and 9 light up 6), so there are only $3! \times 3!$ possible ways to match wires to the segmented display. And sorting out which of those 36 options is actually rather easy.

Our first task, however, is simply to count the number of 1s, 4s, 7s and 8s in the outputs, so simply a job of counting lengths and summing the lengths for 2 (a 1), 3 (a 7), 4 (a 4) and 7 (an 8).

In [1]:
from collections import Counter


def count_1478(lines: list[str]) -> int:
    lengths = Counter()
    for line in lines:
        output = line.partition("|")[-1]
        lengths += Counter(len(num) for num in output.split())
    return lengths[2] + lengths[3] + lengths[4] + lengths[7]


test_notes = """\
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce
""".splitlines()
assert count_1478(test_notes) == 26


In [2]:
import aocd

notes = aocd.get_data(day=8, year=2021).splitlines()
print("Part 1:", count_1478(notes))


Part 1: 362


# Decoding the signals

Now, for the real work of decoding the signals. There are two different ways you could go about this.

## Logical deduction

First, we could use logical deduction to figure out what pattern maps to what digit. We already know what patterns map to 1, 4, 7 and 8. We can use the first three of those to find the rest.

- The signal for 1 gives us two possibilities for the right-hand side segments of the display.
- The signal for 7 shares those two segments and adds one more, so we know that the 3rd is the top segment.
- The segments for 4 share two with the display for 1: the middle horizontal segment and the top left vertical segment.

Now look at the segments for the numbers 2, 3 and 5. I've marked the segments shared with the 1, 4 and 7 number in green:

<pre><code>
  2:      3:      5:
 <span style="color: green">aaaa    aaaa    aaaa</span>
.    <span style="color: green">c</span>  .    <span style="color: green">c  b</span>    .
.    <span style="color: green">c</span>  .    <span style="color: green">c  b</span>    .
 <span style="color: green">dddd    dddd    dddd</span>
e    .  .    <span style="color: green">f</span>  .    <span style="color: green">f</span>
e    .  .    <span style="color: green">f</span>  .    <span style="color: green">f</span>
 gggg    gggg    gggg
</code></pre>

Of these three, the 5 display shares the segments for `1`, `4` and `7` with only _one_ segment from `1` and `7` missing, and it adds the bottom segment. We already know what segment is the top (`a` in the above rendering), and what two signals will light up the top left and middle segments (`b` and `d`), as those are the two segments in the `4` display that the `1` display doesn't share. That leaves the two segments from the `1` display, one of which will be part of this display for a `5`. Just one of the input signals will match that combination, as the 2 and 3 have other overlap with those 3 inputs. Finding which of the two lets us nail down the exact combination for the right-hand side segments, as well as figure out what signal lights up the bottom segment. We can do the same trick with the display for 3 and the 2 unique segments in the number 4, after which the only remaining segment to deduce is the bottom right.

You can make the above decuctions using sets; the unique segments for 4 and 7 are found by removing the segments for the number 1, etc.

## Generating all possible signal patterns

Another approach would be to just generate all possible combinations of signals. There are 7 segments, so there are $7!$ or 5040 possible arrangemnts of segment signals, something we can trivially generate up front. Put these into a dictionary where the keys are a (frozen, immutable) set of the 10 different signal patterns, and the value is another dictionary mapping each signal pattern to the right digit. The segment signals themselves are the 7 segment letters as (frozen) sets. Use a `frozenset()` for these so they can be used as keys in the dictionaries.

Note that with this approach we don't even need to know what letter maps to what segment in the display! Any segment ordering from the specific permutation is encoded in the 10 unique `frozenset` patterns and not stored separately.

I picked the latter option; it's trivial and fast, giving us the answer in $O(n)$ time.


In [3]:
from __future__ import annotations

from itertools import permutations
from typing import Final, Iterable, Literal


SEGMENT = Literal["abcdefg"]
SEGMENT_PATTERN = frozenset[SEGMENT]  #           length between 2 and 7
SEGMENT_MAP = dict[SEGMENT_PATTERN, int]  #       always length 10
OBSERVED_PATTERNS = frozenset[SEGMENT_PATTERN]  # always length 10

# segments used for each digit, as indices
DIGIT_SEGMENTS: Final[tuple[tuple[int]]] = (
    (0, 1, 2, 4, 5, 6),  #    0
    (2, 5),  #                1
    (0, 2, 3, 4, 6),  #       2
    (0, 2, 3, 5, 6),  #       3
    (1, 2, 3, 5),  #          4
    (0, 1, 3, 5, 6),  #       5
    (0, 1, 3, 4, 5, 6),  #    6
    (0, 2, 5),  #             7
    (0, 1, 2, 3, 4, 5, 6),  # 8
    (0, 1, 2, 3, 5, 6),  #    9
)
SEGMENTS: Final[str] = "abcdefg"


def segmentmap_for_permutation(combo: tuple[SEGMENT]) -> SEGMENT_MAP:
    """Generate the segments-to-digits map for a given permunation of abcdefg"""
    return {
        frozenset(combo[seg] for seg in segs): digit
        for digit, segs in enumerate(DIGIT_SEGMENTS)
    }


SIGNAL_COMBINATIONS: Final[dict[OBSERVED_PATTERNS, SEGMENT_MAP]] = {
    frozenset(segmap): segmap
    for segmap in map(segmentmap_for_permutation, permutations(SEGMENTS))
}


def decode_notes(notes: list[str]) -> Iterable[int]:
    for line in notes:
        patterns, _, numbers = line.partition("|")
        observed_patterns = frozenset(frozenset(sig) for sig in patterns.split())
        segment_map = SIGNAL_COMBINATIONS[observed_patterns]
        displayed = 0
        for segments in numbers.split():
            displayed = displayed * 10 + segment_map[frozenset(segments)]
        yield displayed


test_note = (
    "acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | "
    "cdfeb fcadb cdfeb cdbaf"
)
assert next(decode_notes([test_note])) == 5353

assert sum(decode_notes(test_notes)) == 61229


In [4]:
print("Part 2:", sum(decode_notes(notes)))


Part 2: 1020159


In [5]:
import time
from secrets import token_hex
from typing import Any

import IPython.display as display


# SVG template, for a single digit display.
SEGMENT_SVG = """\
<?xml version="1.0" encoding="UTF-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"
height="100" viewBox="0 0 {vbwidth} 20"
stroke="#FFF" stroke-width=".25" fill="#F00">
<defs><path id="h" d="m2 2 1 1h6l1-1 -1-1h-6"/>
<path id="v" d="m2 2-1 1v6l1 1 1-1v-6"/></defs>
<symbol id="bg" fill="#DDD"><use id="a" xlink:href="#h"/><use id="b" xlink:href="#v"/>
<use id="c" xlink:href="#v" x="8"/><use id="d" xlink:href="#h" y="8"/>
<use id="e" xlink:href="#v" y="8"/><use id="f" xlink:href="#v" x="8" y="8"/>
<use id="g" xlink:href="#h" y="16"/></symbol>
<symbol id="d1"><use xlink:href="#bg"/><use xlink:href="#c"/>
<use xlink:href="#f"/></symbol>
<symbol id="d7"><use xlink:href="#d1"/><use xlink:href="#a"/></symbol>
<symbol id="d0"><use xlink:href="#d7"/><use xlink:href="#b"/><use xlink:href="#e"/>
<use xlink:href="#g"/></symbol>
<symbol id="d2"><use xlink:href="#bg"/><use xlink:href="#a"/><use xlink:href="#c"/>
<use xlink:href="#d"/><use xlink:href="#e"/><use xlink:href="#g"/></symbol>
<symbol id="d3"><use xlink:href="#d7"/><use xlink:href="#d"/><use xlink:href="#g"/>
</symbol>
<symbol id="d4"><use xlink:href="#bg"/><use xlink:href="#b"/><use xlink:href="#c"/>
<use xlink:href="#d"/><use xlink:href="#f"/></symbol>
<symbol id="d5"><use xlink:href="#bg"/><use xlink:href="#a"/><use xlink:href="#b"/>
<use xlink:href="#d"/><use xlink:href="#f"/><use xlink:href="#g"/></symbol>
<symbol id="d6"><use xlink:href="#d5"/><use xlink:href="#e"/></symbol>
<symbol id="d8"><use xlink:href="#d0"/><use xlink:href="#d"/></symbol>
<symbol id="d9"><use xlink:href="#d5"/><use xlink:href="#c"/></symbol>
{digits}
</svg>
"""

DIGIT_SVG = """<use xlink:href="#d{digit}" x="{pos}"/>"""
DIGIT_WIDTH = 12


class SevenSegmentNumber(display.DisplayObject):
    def __init__(self, digits: int = 1, value: int = 0, metadata: Any = None) -> None:
        super().__init__(metadata=metadata)
        self._display_id = token_hex(8)
        self._digits = digits
        self._value = value
        self._render_digits()

    @property
    def value(self) -> int:
        return self._value

    @value.setter
    def value(self, value: int) -> None:
        self._value = value
        self._render_digits()
        self.update()

    @property
    def digits(self) -> int:
        return self._digits

    @digits.setter
    def digits(self, digits: int) -> None:
        self._digits = digits
        self._render_digits()
        self.update()

    def _render_digits(self) -> None:
        value, digits = self._value, []
        for _ in range(self._digits):
            digits.append(value % 10)
            value //= 10
        digit_svg = [
            DIGIT_SVG.format(digit=digit, pos=i * DIGIT_WIDTH)
            for i, digit in enumerate(reversed(digits))
        ]
        self.data = SEGMENT_SVG.format(
            vbwidth=DIGIT_WIDTH * self._digits, digits="\n".join(digit_svg)
        )

    def _repr_svg_(self):
        return self._data_and_metadata()

    def display(self):
        display.display(self, display_id=self._display_id)

    def update(self):
        display.display(self, display_id=self._display_id, update=True)
        

def animate_decoding() -> None:
    markdown_text = display.Markdown("## &nbsp;")
    text_display = display.display(markdown_text, display_id=token_hex(8))
    decoded, total = SevenSegmentNumber(digits=4), SevenSegmentNumber(digits=7)
    decoded.display()
    total.display()
    for i, (note, value) in enumerate(zip(notes, decode_notes(notes), strict=True), 1):
        message = f"## {i} / {len(notes)}: `{note.partition('|')[0].strip()}`"
        markdown_text.data = message
        text_display.update(markdown_text)
        decoded.value = value
        total.value += value
        time.sleep(0.20)

# Animation

Here's a little animation that plays in the notebook, showing the numbers being 'decoded' with the running total:

In [7]:
animate_decoding()

## 51 / 200: `dcefabg bfaed feg ebgfa eg egad cgdebf cgfab degbfa acdefb`